# Task 3-2
Нужно, используя Spark: 
- найти явку (%) по всем регионам, результат отсортировать по убыванию 
- выбрать любимого кандидата и найти тот избиратльный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек) 
- найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна 
- посчитать дисперсию по явке для каждого региона (учитывать УИК) 
- для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

Результаты принимаются в виде Jupyter Notebook, Spark Notebook или исходных файлов на Scala.

In [1]:
import pyspark
import os

from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Возможно нужно будет явно указать Spark'у откуда брать интерпретатор Python для worker'ов
os.environ["PYSPARK_PYTHON"]="python3"

In [2]:
spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

- 'Регион'
- 'Район'
- 'Уик'
- 'Число избирателей, включенных в список избирателей',                                        
- 'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
- 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно',
- 'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования',
- 'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования',
- 'Число погашенных избирательных бюллетеней',
- 'Число избирательных бюллетеней в переносных ящиках для голосования',
- 'Число бюллетеней в стационарных ящиках для голосования',
- 'Число недействительных избирательных бюллетеней',
- 'Число действительных избирательных бюллетеней',
- 'Число утраченных избирательных бюллетеней',
- 'Число избирательных бюллетеней, не учтенных при получении',
- 'Бабурин Сергей Николаевич', 
- 'Грудинин Павел Николаевич',
- 'Жириновский Владимир Вольфович', 
- 'Путин Владимир Владимирович',
- 'Собчак Ксения Анатольевна', 
- 'Сурайкин Максим Александрович',
- 'Титов Борис Юрьевич', 
- 'Явлинский Григорий Алексеевич'

## Загрузка данных и схема данных

In [3]:
filename = 'local_df_1.csv'

schema= StructType([
            StructField("region",              StringType(), False), #1
            StructField("tik",                 StringType(), False), #2
            StructField("uik",                 StringType(), False), #3
            StructField("voters_count",        IntegerType(), False),#4
            StructField("blanks_received",     IntegerType(), False),#5
            StructField("gived_voters_blanks", IntegerType(), False),#6
            StructField("give_elec_day_blanks",IntegerType(), False),#7
            StructField("not_elec_day_blanks", IntegerType(), False),#8
            StructField("cancel_blanks",       IntegerType(), False),#9
            StructField("transfer_box_blanks", IntegerType(), False),#10
            StructField("stat_box_blanks",     IntegerType(), False),#11
            StructField("invalid_blanks",      IntegerType(), False),#12
            StructField("valid_blanks",        IntegerType(), False),#13
            StructField("lost_blanks",         IntegerType(), False),#14
            StructField("uncount_blanks",      IntegerType(), False),#15
            StructField("babur",               IntegerType(), False),#16
            StructField("grud",                IntegerType(), False),#17
            StructField("jirin",               IntegerType(), False),#18
            StructField("putin",               IntegerType(), False),#19
            StructField("soob",                IntegerType(), False),#20
            StructField("sur",                 IntegerType(), False),#21
            StructField("titov",               IntegerType(), False),#22
            StructField("yavl",                IntegerType(), False)]#23
        )

df = spark.read.csv(filename, schema=schema)

## Подсчет явки в процентах

In [4]:
region_statistic = df.groupBy('region').agg(
    F.sum('voters_count').alias('voters_count'),
    F.sum('stat_box_blanks').alias('stat_box_blanks'),
    F.sum('transfer_box_blanks').alias('transfer_box_blanks'),
).dropna().select(
    F.col('region'),
    (
        (F.col('stat_box_blanks')+
         F.col('transfer_box_blanks'))/
        F.col('voters_count')
    ).alias('appearance')
).sort('appearance', ascending=False).show()  

+--------------------+------------------+
|              region|        appearance|
+--------------------+------------------+
|Территория за пре...|0.9801821236184206|
|     Республика Тыва|0.9362542974951742|
|Ямало-Ненецкий ав...|0.9187213306617982|
|Кабардино-Балкарс...| 0.916978754085207|
|Чеченская Республика|0.9150929703298174|
|Республика Северн...|0.8993704831730909|
| Республика Дагестан|0.8744351419820495|
|Карачаево-Черкесс...|0.8734380939009899|
| Кемеровская область|0.8307556958224817|
|Чукотский автоном...|0.8226051697921947|
|Республика Ингушетия|0.8195937165135314|
|    Брянская область| 0.796597680757152|
|   Тюменская область|0.7888296737637491|
| Республика Мордовия|0.7783648793754097|
|  Краснодарский край|0.7783466166202626|
|Республика Татарс...|0.7736526548202729|
|Чувашская Республ...|0.7621061419670118|
|Республика Башкор...|0.7542120065745193|
|  Республика Бурятия|0.7519133382510174|
|Республика Адыгея...| 0.743059515324305|
+--------------------+------------

## Выбрать любимого кандидата и найти тот избиратльный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)

In [5]:
favorite = 'grud'
favorite_candidat_votes = df.select(
    F.col('region'),
    F.col('tik'),
    F.col('uik'),
    (F.col(favorite)/F.col('voters_count')).alias('percent')
).dropna().sort('percent', ascending=False).show(1)

+---------------+--------------------+---------+-------+
|         region|                 tik|      uik|percent|
+---------------+--------------------+---------+-------+
|Приморский край|Владивосток, Фрун...|УИК №5924|    0.8|
+---------------+--------------------+---------+-------+
only showing top 1 row



## Найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна

In [6]:
df.select(
    F.col('region'),
    F.col('tik'),
    F.col('voters_count'),
    F.col('stat_box_blanks'),
    F.col('transfer_box_blanks')
).groupBy('region', 'tik').agg(
    F.sum('voters_count').alias('voters_count'),
    F.sum('stat_box_blanks').alias('stat_box_blanks'),
    F.sum('transfer_box_blanks').alias('transfer_box_blanks'),
).select(
    F.col('region'),
    (
        (F.col('stat_box_blanks')+
         F.col('transfer_box_blanks'))/
        F.col('voters_count')
    ).alias('appearance')
).dropna().groupBy('region').agg(
    F.max('appearance').alias('max_ap'),
    F.min('appearance').alias('min_ap')
).select(
    F.col('region'),
    (F.col('max_ap')-F.col('min_ap')).alias('diff_ap')
).sort('diff_ap', ascending=False).show(1)


+--------------------+-------------------+
|              region|            diff_ap|
+--------------------+-------------------+
|Архангельская обл...|0.49859164840684894|
+--------------------+-------------------+
only showing top 1 row



## Посчитать дисперсию по явке для каждого региона (учитывать УИК)

In [7]:
df.groupBy('region').agg(
    F.stddev((F.col('stat_box_blanks')+
         F.col('transfer_box_blanks'))/
        F.col('voters_count')
    ).alias('std_appearance')
).sort('std_appearance', ascending=False).show()


+--------------------+-------------------+
|              region|     std_appearance|
+--------------------+-------------------+
| Сахалинская область|0.20036478651651426|
|     Камчатский край| 0.1667941105864646|
|  Мурманская область| 0.1658339344574538|
|     Приморский край|0.16489945928071562|
|Республика Адыгея...|0.16280291848500575|
|   Самарская область|0.15587712308673865|
| Саратовская область|0.15308940168788865|
| Республика Калмыкия|0.15204196427553218|
| Воронежская область|0.14877889931086916|
|Республика Татарс...|0.14116631888504572|
|    Липецкая область|0.14028570150699685|
|Архангельская обл...|0.13965138254034642|
| Магаданская область|0.13939278127769808|
|  Краснодарский край| 0.1386015769911046|
|Белгородская область|0.13686039659009372|
|    Хабаровский край|0.13677851570331015|
| Ставропольский край|0.13318004559436392|
|  Республика Бурятия| 0.1322534539911989|
|  Ростовская область|0.13043726913960704|
|Чувашская Республ...|0.13003319051362872|
+----------

## Для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат¶

In [8]:
candidates = ["babur", "grud", "jirin", "putin", "soob", "sur", "titov", "yavl"]

frame_candidates = [(cand, df.select(
    (
        F.ceil(100*F.col(cand)/
        F.col('voters_count'))).alias('result')
).groupBy('result').agg(F.count('result')
).dropna().sort('count(result)', ascending=False)) for cand in candidates]

In [9]:
def print_cand(cand):
    name, frame = cand
    print(name)
    frame.show()
list(map(print_cand, frame_candidates))

babur
+------+-------------+
|result|count(result)|
+------+-------------+
|     1|        75677|
|     0|        19508|
|     2|         4449|
|     3|          514|
|     4|          155|
|     5|           70|
|     6|           26|
|     7|           20|
|     8|           18|
|    10|            9|
|     9|            7|
|    11|            4|
|    15|            2|
|    12|            2|
|    14|            2|
|    17|            2|
|    20|            2|
|    18|            1|
|    24|            1|
|    13|            1|
+------+-------------+
only showing top 20 rows

grud
+------+-------------+
|result|count(result)|
+------+-------------+
|     8|        13380|
|     7|        12510|
|     9|        11358|
|     6|         9697|
|    10|         8289|
|     5|         7122|
|    11|         5821|
|     4|         5406|
|     3|         4410|
|    12|         4097|
|     2|         3807|
|    13|         3110|
|    14|         2337|
|     1|         1599|
|    15|         158

[None, None, None, None, None, None, None, None]